In [26]:
import os
import time
import json
import torch
from torch import nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# Пути
DATA_PATH = r"C:\Users\safar\OneDrive\Рабочий стол\elbrus\my_project_2_1\data\birds\CUB_200_2011\images"
BASE_DIR = r"C:\Users\safar\OneDrive\Рабочий стол\elbrus\my_project_2_1"
MODELS_DIR = os.path.join(BASE_DIR, "models")
SAVE_PATH = os.path.join(MODELS_DIR, "densenet_birds_finetuned.pt")
LOG_PATH = os.path.join(MODELS_DIR, "train_log_finetuned.json")
CONFUSION_PATH = os.path.join(MODELS_DIR, "confusion_finetuned.json")
EPOCHS = 3
BATCH_SIZE = 16

# Трансформации
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Датасет и DataLoader
dataset = datasets.ImageFolder(DATA_PATH, transform=transform)

# Разделение на train и valid
train_size = int(0.8 * len(dataset))  # 80% для обучения
valid_size = len(dataset) - train_size  # 20% для валидации
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

classes = dataset.classes
print("Классы:", classes)

# Модель (DenseNet-121)
model = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)

# Заморозка всех слоев
for param in model.parameters():
    param.requires_grad = False

# Fine-tuning: Создаем новый классификатор с дополнительными слоями
num_classes = len(classes)
model.classifier = nn.Sequential(
    nn.Linear(model.classifier.in_features, 512),  # Полносвязный слой
    nn.BatchNorm1d(512),                           # Нормализация по батчам
    nn.ReLU(),                                     # Активация ReLU
    nn.Dropout(0.5),                               # Dropout для регуляризации
    nn.Linear(512, num_classes)                    # Выходной слой
)

# Устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Функция потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)

# Логирование
train_losses, valid_losses = [], []
train_accuracies, valid_accuracies = [], []

start_time = time.time()

# Обучение
for epoch in range(EPOCHS):
    # Train
    model.train()
    running_loss = 0
    y_true_train, y_pred_train = [], []

    for images, labels in tqdm(train_loader, desc=f"Train Эпоха {epoch+1}/{EPOCHS}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Для метрик
        preds = torch.argmax(outputs, dim=1)
        y_true_train.extend(labels.cpu().tolist())
        y_pred_train.extend(preds.cpu().tolist())

    train_loss = running_loss / len(train_loader)
    train_accuracy = accuracy_score(y_true_train, y_pred_train)
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # Validation
    model.eval()
    running_loss = 0
    y_true_valid, y_pred_valid = [], []

    with torch.no_grad():  # Отключаем вычисление градиентов
        for images, labels in tqdm(valid_loader, desc=f"Valid Эпоха {epoch+1}/{EPOCHS}"):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            # Для метрик
            preds = torch.argmax(outputs, dim=1)
            y_true_valid.extend(labels.cpu().tolist())
            y_pred_valid.extend(preds.cpu().tolist())

    valid_loss = running_loss / len(valid_loader)
    valid_accuracy = accuracy_score(y_true_valid, y_pred_valid)
    valid_losses.append(valid_loss)
    valid_accuracies.append(valid_accuracy)

    # Вывод метрик
    print(f"[{epoch+1}] Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
    print(f"[{epoch+1}] Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_accuracy:.4f}")

# Время
total_time = time.time() - start_time
print("✅ Обучение завершено.")
print(f"⏱️ Время обучения: {round(total_time / 60, 2)} минут")

# Создание директории models, если она не существует
os.makedirs(MODELS_DIR, exist_ok=True)

# Сохраняем модель
torch.save(model.state_dict(), SAVE_PATH)
print("💾 Модель сохранена в:", SAVE_PATH)

# Сохраняем лог
train_log = {
    "train_losses": train_losses,
    "train_accuracies": train_accuracies,
    "valid_losses": valid_losses,
    "valid_accuracies": valid_accuracies,
    "training_time": total_time
}
with open(LOG_PATH, "w") as f:
    json.dump(train_log, f)
print("📝 Лог обучения сохранён.")

# Сохраняем confusion данные
conf_data = {
    "true": y_true_valid,
    "pred": y_pred_valid
}
with open(CONFUSION_PATH, "w") as f:
    json.dump(conf_data, f)
print("🧩 Данные для confusion matrix сохранены.")

Классы: ['001.Black_footed_Albatross', '002.Laysan_Albatross', '003.Sooty_Albatross', '004.Groove_billed_Ani', '005.Crested_Auklet', '006.Least_Auklet', '007.Parakeet_Auklet', '008.Rhinoceros_Auklet', '009.Brewer_Blackbird', '010.Red_winged_Blackbird', '011.Rusty_Blackbird', '012.Yellow_headed_Blackbird', '013.Bobolink', '014.Indigo_Bunting', '015.Lazuli_Bunting', '016.Painted_Bunting', '017.Cardinal', '018.Spotted_Catbird', '019.Gray_Catbird', '020.Yellow_breasted_Chat', '021.Eastern_Towhee', '022.Chuck_will_Widow', '023.Brandt_Cormorant', '024.Red_faced_Cormorant', '025.Pelagic_Cormorant', '026.Bronzed_Cowbird', '027.Shiny_Cowbird', '028.Brown_Creeper', '029.American_Crow', '030.Fish_Crow', '031.Black_billed_Cuckoo', '032.Mangrove_Cuckoo', '033.Yellow_billed_Cuckoo', '034.Gray_crowned_Rosy_Finch', '035.Purple_Finch', '036.Northern_Flicker', '037.Acadian_Flycatcher', '038.Great_Crested_Flycatcher', '039.Least_Flycatcher', '040.Olive_sided_Flycatcher', '041.Scissor_tailed_Flycatcher', 

Valid Эпоха 1/3: 100%|██████████| 148/148 [00:28<00:00,  5.15it/s]


[1] Train Loss: 3.1353, Train Accuracy: 0.2857
[1] Valid Loss: 1.7167, Valid Accuracy: 0.5534


Valid Эпоха 2/3: 100%|██████████| 148/148 [00:36<00:00,  4.06it/s]


[2] Train Loss: 1.7430, Train Accuracy: 0.5285
[2] Valid Loss: 1.4360, Valid Accuracy: 0.6077


Valid Эпоха 3/3: 100%|██████████| 148/148 [00:36<00:00,  4.00it/s]


[3] Train Loss: 1.4113, Train Accuracy: 0.6107
[3] Valid Loss: 1.3173, Valid Accuracy: 0.6327
✅ Обучение завершено.
⏱️ Время обучения: 9.07 минут
💾 Модель сохранена в: C:\Users\safar\OneDrive\Рабочий стол\elbrus\my_project_2_1\models\densenet_birds_finetuned.pt
📝 Лог обучения сохранён.
🧩 Данные для confusion matrix сохранены.


In [ ]:
# import os
# import time
# import json
# import torch
# from torch import nn
# from torchvision import datasets, transforms, models
# from torch.utils.data import DataLoader, random_split
# from sklearn.metrics import accuracy_score
# from tqdm import tqdm

# # Пути
# DATA_PATH = r"C:\Users\safar\OneDrive\Рабочий стол\elbrus\my_project_2_1\data\birds\CUB_200_2011\images"
# # SAVE_PATH = "my_project_2_1\models\resnet_birds.pt"
# # LOG_PATH = "\my_project_2_1\models\train_log_2.json"
# # CONFUSION_PATH = "\my_project_2_1\models\confusion2.json"
# BASE_DIR = r"C:\Users\safar\OneDrive\Рабочий стол\elbrus\my_project_2_1"
# MODELS_DIR = os.path.join(BASE_DIR, "models")
# SAVE_PATH = os.path.join(MODELS_DIR, "resnet_birds.pt")
# LOG_PATH = os.path.join(MODELS_DIR, "train_log_2.json")
# CONFUSION_PATH = os.path.join(MODELS_DIR, "confusion2.json")
# EPOCHS = 3
# BATCH_SIZE = 16



# # Трансформации
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406],
#                          [0.229, 0.224, 0.225])
# ])

# # Датасет и DataLoader
# dataset = datasets.ImageFolder(DATA_PATH, transform=transform)

# # Разделение на train и valid
# train_size = int(0.8 * len(dataset))  # 80% для обучения
# valid_size = len(dataset) - train_size  # 20% для валидации
# train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

# classes = dataset.classes
# print("Классы:", classes)

# # Модель (DenseNet-121)
# model = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)

# # Заморозка всех слоев
# for param in model.parameters():
#     param.requires_grad = False

# # Замена последнего слоя
# num_classes = len(classes)
# model.classifier = nn.Linear(model.classifier.in_features, num_classes)

# # Устройство
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

# # Функция потерь и оптимизатор
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)

# # Логирование
# train_losses, valid_losses = [], []
# train_accuracies, valid_accuracies = [], []

# start_time = time.time()

# # Обучение
# for epoch in range(EPOCHS):
#     # Train
#     model.train()
#     running_loss = 0
#     y_true_train, y_pred_train = [], []

#     for images, labels in tqdm(train_loader, desc=f"Train Эпоха {epoch+1}/{EPOCHS}"):
#         images, labels = images.to(device), labels.to(device)

#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#         # Для метрик
#         preds = torch.argmax(outputs, dim=1)
#         y_true_train.extend(labels.cpu().tolist())
#         y_pred_train.extend(preds.cpu().tolist())

#     train_loss = running_loss / len(train_loader)
#     train_accuracy = accuracy_score(y_true_train, y_pred_train)
#     train_losses.append(train_loss)
#     train_accuracies.append(train_accuracy)

#     # Validation
#     model.eval()
#     running_loss = 0
#     y_true_valid, y_pred_valid = [], []

#     with torch.no_grad():  # Отключаем вычисление градиентов
#         for images, labels in tqdm(valid_loader, desc=f"Valid Эпоха {epoch+1}/{EPOCHS}"):
#             images, labels = images.to(device), labels.to(device)

#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             running_loss += loss.item()

#             # Для метрик
#             preds = torch.argmax(outputs, dim=1)
#             y_true_valid.extend(labels.cpu().tolist())
#             y_pred_valid.extend(preds.cpu().tolist())

#     valid_loss = running_loss / len(valid_loader)
#     valid_accuracy = accuracy_score(y_true_valid, y_pred_valid)
#     valid_losses.append(valid_loss)
#     valid_accuracies.append(valid_accuracy)

#     # Вывод метрик
#     print(f"[{epoch+1}] Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
#     print(f"[{epoch+1}] Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_accuracy:.4f}")

# # Время
# total_time = time.time() - start_time
# print("✅ Обучение завершено.")
# print(f"⏱️ Время обучения: {round(total_time / 60, 2)} минут")

# # Сохраняем модель
# os.makedirs("models", exist_ok=True)
# torch.save(model.state_dict(), SAVE_PATH)
# print("💾 Модель сохранена в:", SAVE_PATH)

# # Сохраняем лог
# train_log = {
#     "train_losses": train_losses,
#     "train_accuracies": train_accuracies,
#     "valid_losses": valid_losses,
#     "valid_accuracies": valid_accuracies,
#     "training_time": total_time
# }
# with open(LOG_PATH, "w") as f:
#     json.dump(train_log, f)
# print("📝 Лог обучения сохранён.")

# # Сохраняем confusion данные
# conf_data = {
#     "true": y_true_valid,
#     "pred": y_pred_valid
# }
# with open(CONFUSION_PATH, "w") as f:
#     json.dump(conf_data, f)
# print("🧩 Данные для confusion matrix сохранены.")

Классы: ['001.Black_footed_Albatross', '002.Laysan_Albatross', '003.Sooty_Albatross', '004.Groove_billed_Ani', '005.Crested_Auklet', '006.Least_Auklet', '007.Parakeet_Auklet', '008.Rhinoceros_Auklet', '009.Brewer_Blackbird', '010.Red_winged_Blackbird', '011.Rusty_Blackbird', '012.Yellow_headed_Blackbird', '013.Bobolink', '014.Indigo_Bunting', '015.Lazuli_Bunting', '016.Painted_Bunting', '017.Cardinal', '018.Spotted_Catbird', '019.Gray_Catbird', '020.Yellow_breasted_Chat', '021.Eastern_Towhee', '022.Chuck_will_Widow', '023.Brandt_Cormorant', '024.Red_faced_Cormorant', '025.Pelagic_Cormorant', '026.Bronzed_Cowbird', '027.Shiny_Cowbird', '028.Brown_Creeper', '029.American_Crow', '030.Fish_Crow', '031.Black_billed_Cuckoo', '032.Mangrove_Cuckoo', '033.Yellow_billed_Cuckoo', '034.Gray_crowned_Rosy_Finch', '035.Purple_Finch', '036.Northern_Flicker', '037.Acadian_Flycatcher', '038.Great_Crested_Flycatcher', '039.Least_Flycatcher', '040.Olive_sided_Flycatcher', '041.Scissor_tailed_Flycatcher', 

Valid Эпоха 1/3: 100%|██████████| 148/148 [00:36<00:00,  4.10it/s]


[1] Train Loss: 3.5874, Train Accuracy: 0.2615
[1] Valid Loss: 2.2539, Valid Accuracy: 0.4771


Valid Эпоха 2/3: 100%|██████████| 148/148 [00:37<00:00,  3.92it/s]


[2] Train Loss: 1.7912, Train Accuracy: 0.5930
[2] Valid Loss: 1.6980, Valid Accuracy: 0.5814


Valid Эпоха 3/3: 100%|██████████| 148/148 [00:38<00:00,  3.84it/s]


[3] Train Loss: 1.2607, Train Accuracy: 0.7048
[3] Valid Loss: 1.5190, Valid Accuracy: 0.6183
✅ Обучение завершено.
⏱️ Время обучения: 9.79 минут
💾 Модель сохранена в: C:\Users\safar\OneDrive\Рабочий стол\elbrus\my_project_2_1\models\resnet_birds.pt
📝 Лог обучения сохранён.
🧩 Данные для confusion matrix сохранены.


In [ ]:
device

In [16]:
# import os
# import time
# import json
# import torch
# from torch import nn
# from torchvision import datasets, transforms, models
# from torch.utils.data import DataLoader, random_split
# from sklearn.metrics import accuracy_score
# from tqdm import tqdm

In [17]:
# # Пути
# DATA_PATH = r"C:\Users\safar\OneDrive\Рабочий стол\elbrus\my_project_2_1\data\birds\CUB_200_2011\images"
# SAVE_PATH = "models/resnet_birds.pt"
# LOG_PATH = "models/train_log_2.json"
# CONFUSION_PATH = "models/confusion2.json"
# EPOCHS = 3
# BATCH_SIZE = 16

In [18]:
# # Трансформации
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406],
#                          [0.229, 0.224, 0.225])
# ])

In [19]:
# # Датасет и DataLoader
# dataset = datasets.ImageFolder(DATA_PATH, transform=transform)

In [20]:
# # Разделение на train и valid
# train_size = int(0.8 * len(dataset))  # 80% для обучения
# valid_size = len(dataset) - train_size  # 20% для валидации
# train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [21]:
# classes = dataset.classes
# print("Классы:", classes)

In [22]:
# # Получить список всех доступных моделей
# available_models = [name for name in dir(models) if not name.startswith("_")]
# print("Доступные модели:", available_models)

In [23]:
#model = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)

In [24]:
# model = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
# for param in model.parameters():
#     param.requires_grad = False
# model.fc = nn.Linear(model.fc.in_features, len(classes))

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

In [25]:
# # Модель
# model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
# for param in model.parameters():
#     param.requires_grad = False
# model.fc = nn.Linear(model.fc.in_features, len(classes))

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

# # Логирование
# train_losses, valid_losses = [], []
# train_accuracies, valid_accuracies = [], []

# start_time = time.time()

# # Обучение
# for epoch in range(EPOCHS):
#     # Train
#     model.train()
#     running_loss = 0
#     y_true_train, y_pred_train = [], []

#     for images, labels in tqdm(train_loader, desc=f"Train Эпоха {epoch+1}/{EPOCHS}"):
#         images, labels = images.to(device), labels.to(device)

#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#         # Для метрик
#         preds = torch.argmax(outputs, dim=1)
#         y_true_train.extend(labels.cpu().tolist())
#         y_pred_train.extend(preds.cpu().tolist())

#     train_loss = running_loss / len(train_loader)
#     train_accuracy = accuracy_score(y_true_train, y_pred_train)
#     train_losses.append(train_loss)
#     train_accuracies.append(train_accuracy)

#     # Validation
#     model.eval()
#     running_loss = 0
#     y_true_valid, y_pred_valid = [], []

#     with torch.no_grad():  # Отключаем вычисление градиентов
#         for images, labels in tqdm(valid_loader, desc=f"Valid Эпоха {epoch+1}/{EPOCHS}"):
#             images, labels = images.to(device), labels.to(device)

#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             running_loss += loss.item()

#             # Для метрик
#             preds = torch.argmax(outputs, dim=1)
#             y_true_valid.extend(labels.cpu().tolist())
#             y_pred_valid.extend(preds.cpu().tolist())

#     valid_loss = running_loss / len(valid_loader)
#     valid_accuracy = accuracy_score(y_true_valid, y_pred_valid)
#     valid_losses.append(valid_loss)
#     valid_accuracies.append(valid_accuracy)

#     # Вывод метрик
#     print(f"[{epoch+1}] Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
#     print(f"[{epoch+1}] Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_accuracy:.4f}")

# # Время
# total_time = time.time() - start_time
# print("✅ Обучение завершено.")
# print(f"⏱️ Время обучения: {round(total_time / 60, 2)} минут")

# # Сохраняем модель
# os.makedirs("models", exist_ok=True)
# torch.save(model.state_dict(), SAVE_PATH)
# print("💾 Модель сохранена в:", SAVE_PATH)

# # Сохраняем лог
# train_log = {
#     "train_losses": train_losses,
#     "train_accuracies": train_accuracies,
#     "valid_losses": valid_losses,
#     "valid_accuracies": valid_accuracies,
#     "training_time": total_time
# }
# with open(LOG_PATH, "w") as f:
#     json.dump(train_log, f)
# print("📝 Лог обучения сохранён.")

# # Сохраняем confusion данные
# conf_data = {
#     "true": y_true_valid,
#     "pred": y_pred_valid
# }
# with open(CONFUSION_PATH, "w") as f:
#     json.dump(conf_data, f)
# print("🧩 Данные для confusion matrix сохранены.")